In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import string

from clean_data import*
from add_summary import*
from find_category import*

In [2]:
candidate=DataFrame(columns=["id", "name","state_fk","date_birth", "place_birth", "position", "URL_photo", "party"])

candidate1=pd.DataFrame([[1, "Don Blankenship",1, "March 14, 1950", "Stopover, Kentucky", "Coal magnate", "https://i2.wp.com/woay.tv/wp-content/uploads/2017/05/GTY_don_blankenship_jt_160406_4x3_992.jpg?resize=735%2C400&ssl=1","Republican" ],
                     [2, "Bo Copley",1, "", " Dingess, West Virginia", "","https://heavyeditorial.files.wordpress.com/2017/05/bo-copley-e1494012597984.jpg?quality=65&strip=all&w=780&strip=all","Republican" ],
                     [3, "Evan Jenkins",1, "September 12, 1960", "Huntington, West Virginia","U.S. House of Representatives","https://www.congress.gov/img/member/114_rp_wv_3_jenkins_evan.jpg","Republican"  ],
                     [4, "Joe Manchin",1, " August 24, 1947", "Farmington, West Virginia", "United States Senator", "https://fi.wikipedia.org/wiki/Joe_Manchin#/media/File:Senator_Manchin.jpg", "Democrat"],
                     [5, "Patrick Morrisey",1,"December 21, 1967","Brooklyn, New York City, NY","West Virginia Attorney General since 2013","http://wvmetronews.com/wordpress/wp-content/uploads/2016/02/Patrick-Morrisey.jpg?x43308","Republican"  ],
                     [6, "Jack Newbrough",1, "", "","U.S. Navy veteran","http://s3.amazonaws.com/ogden_images/www.weirtondailytimes.com/images/2017/09/23190845/Newbrough-003-1100x825.jpg" ,"Republican" ],
                     [7, "Paula Jean Swearengin", 1, "born 1973/74","Mullens, West Virginia","","https://cbbba9d899c5f8ae6108-e379174d4d25dcb78174c6cab0a88cea.ssl.cf1.rackcdn.com/1275564-l.jpg", "Democrat"],
                     [8, "Tom Willis",1, "","","","https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/2C512069-7298-4BAC-B9F3-6BFECDC2FB57.jpeg","Republican" ],
                     [9, "Nicholas Freitas", 2, "", "", "","","Republican"],
                     [10, "E. W. Jackson", 2,"","","","","Republican"],
                     [11, "Tim Kaine", 2, "", "", "", "", "Democrat"],
                     [12, "Corey Stewart", 2, "", "", "", "", "Republican"],
                     [13, "Matt Waters", 2, "", "", "", "", "Libertarian"],
                     [14, "Carl Bible", 3, "", "", "", "", "Independent"],
                     [15, "Ted Cruz", 3, "", "", "", "", "Republican"],
                     [16, "Bob McNeil", 3, "", "", "", "", "American Citizen Party"],
                     [17, "Beto O'Rourke", 3, "", "", "", "", "Democrat"]],
                        columns=["id", "name","state_fk","date_birth", "place_birth", "position","URL_photo", "party"])
candidate=candidate.append(candidate1)


In [3]:
all_candidates = []
for index,row in candidate.iterrows():
    all_candidates.append(row['name'])

In [4]:
## script is hardcoded to take in data from the RAW_DATA file
# this function takes in a list of candidates (string form) and a name of the output json file
structure_data(all_candidates, "ALL_DATA.json")

In [ ]:
# this script takes an input file and creates a new json
add_summary("ALL_DATA.json", "ALL_DATA_SUMM.json")

In [ ]:
all_data = pd.read_json("ALL_DATA_SUMM.json")

In [ ]:
## category analysis

In [ ]:
# call script to generate categories and 

all_candidates_str = [x.split() for x in all_candidates]
all_candidates_str = [x.lower() for y in all_candidates_str for x in y]

extra_words = ["like", "it’s", "uh", "going", "that’s", "think", "actually", "kind", "…", "know", "come", "u", "really"
                       ,"mr", "june", "july", "august", "aug", "festival", "theater", "music", "concert", "dance", "performance"
                       , "jazz", "band", "arts", "art","museum","painting", "artist", "exhibition","gallery", "ms", "sculpture"
                       , "tell", "also","thats", "im", "album", "film", "movie", "opera", "cookbook", "orchestra", "play"
                       , "street", "design", "photograph", "drawing", "collection", "street", "design", "direct", "jan", "may","summer"
                       , "season", "production", "park", "oct", "show", "quartet", "series", "may", "star", "feature", "tonight"
                       , "avenue", "village", "production", "summer", "season", "premiere","ballet", "photograph", "drawing", "boat"
                       , "symphony", "shakespeare", "repertory","pop", "perform", "perform", "night", "classical", "guest", "musical"]

extra_words.extend(all_candidates_str)


In [ ]:
corpus_text = []
for index,row in all_data.iterrows():
    corpus_text.append(row['article_text'])

In [ ]:
corpus_titles = []
for index,row in all_data.iterrows():
    corpus_titles.append(row['articles_title'])

In [ ]:
relevant_words_text = find_categories_and_top20(corpus_text, extra_words)

In [ ]:
relevant_words_title = find_categories_and_top20(corpus_titles, extra_words)

In [ ]:
create_lda_page(relevant_words, "lda_no_polit")

In [ ]:
create_lda_page(relevant_words_title, "lda_titles")

In [ ]:
b = [[2*x for y in a] for x in y]
c = [[2*x for x in y] for y in a]
#[[int(x) for x in feet] for x in feet]

In [5]:
all_data = pd.read_json("ALL_DATA.json")

In [14]:
all_data

,article_text,articles_date,articles_link,articles_title,first_name,id,last_name,newspaper_name,state_fk
0,"Huntington, West Virginia (CNN) It was as if a...",27 4 2017,https://www.cnn.com/2017/04/27/politics/joe-ma...,Manchin charts own path in Trump-era West Virg...,Patrick,1,Morrisey,CNN,1
1,The legacy of the Hatfields vs. the McCoys loo...,16 5 2014,https://www.cnn.com/2014/05/16/us/west-virgini...,W. Virginia AG: Middle school concealed allege...,Patrick,2,Morrisey,CNN,1
2,West Virginia's attorney general and state leg...,14 1 2014,https://www.cnn.com/2014/01/14/us/west-virgini...,"More investigations launched as 180,000 West V...",Patrick,3,Morrisey,CNN,1
3,Subscribe to these events on your Google or ...,21 3 2018,https://www.cnn.com/2018/03/06/politics/midter...,What you need to know right now about the 2018...,Patrick,4,Morrisey,CNN,1
4,Washington (CNN) Democratic West Virginia Sen....,7 8 2017,https://www.cnn.com/2017/08/07/politics/joe-ma...,Joe Manchin: 'Don't give a s--t' about attacks,Patrick,5,Morrisey,CNN,1
5,Washington (CNN) President Donald Trump's Frid...,23 3 2018,https://www.cnn.com/2018/03/23/politics/trump-...,Trump's veto threat on spending bill gives amm...,Patrick,6,Morrisey,CNN,1
6,Washington (CNN) Several Republican candidates...,3 1 2018,https://www.cnn.com/2018/01/03/politics/steve-...,Some Bannon-backed Senate hopefuls begin dista...,Patrick,7,Morrisey,CNN,1
7,"Washington (CNN) Democratic incumbents, challe...",5 2 2018,https://www.cnn.com/2018/02/05/politics/senate...,The 10 Senate seats most likely to flip in 201...,Patrick,8,Morrisey,CNN,1
8,(CNN) The West Virginia Supreme Court of Appe...,12 5 2017,https://www.cnn.com/2017/05/12/us/west-virgini...,Court: West Virginia hate crime law doesn't in...,Patrick,9,Morrisey,CNN,1
9,"White Sulphur Springs, West Virginia (CNN) The...",5 4 2018,https://www.cnn.com/2018/04/05/politics/donald...,Trump targets Democratic senator in free-wheel...,Patrick,10,Morrisey,CNN,1


In [ ]:
for index,row in all_data.iterrows():
    print(row["articles_link"])

In [7]:
frames = []
frames.extend(return_dataframes("RAW_DATA/cnn_westvirginia.json","RAW_DATA/jsfoxnews_westvirginia.json","RAW_DATA/nyt_westvirginia.json",1 ))
frames.extend(return_dataframes("RAW_DATA/cnn_virginia.json","RAW_DATA/jsfoxnews_virginia.json","RAW_DATA/nyt_westvirginia.json",2 ))
frames.extend(return_dataframes("RAW_DATA/cnn_texas.json","RAW_DATA/jsfoxnews_texas.json","RAW_DATA/nyt_texas.json",3 ))

# combine all frames
first_data = pd.concat(frames, ignore_index=True)

In [11]:
first_data

,article_text,articles_date,articles_link,articles_title,first_name,last_name,newspaper_name,state_fk
0,"[Huntington, West Virginia (CNN), It was as if...","[Updated 6:14 AM ET, Thu April 27, 2017 ]",https://www.cnn.com/2017/04/27/politics/joe-ma...,[Manchin charts own path in Trump-era West Vir...,Patrick,Morrisey,CNN,1
1,[],[],https://www.cnn.com/videos/bestoftv/2014/01/15...,[],Patrick,Morrisey,CNN,1
2,[The legacy of the Hatfields vs. the McCoys lo...,"[Updated 10:15 PM ET, Fri May 16, 2014 ]",https://www.cnn.com/2014/05/16/us/west-virgini...,[W. Virginia AG: Middle school concealed alleg...,Patrick,Morrisey,CNN,1
3,[West Virginia's attorney general and state le...,"[Updated 9:30 PM ET, Tue January 14, 2014 ]",https://www.cnn.com/2014/01/14/us/west-virgini...,"[More investigations launched as 180,000 West ...",Patrick,Morrisey,CNN,1
4,"[Subscribe to these events on your , Google, ...","[Updated 8:50 AM ET, Wed March 21, 2018]",https://www.cnn.com/2018/03/06/politics/midter...,[What you need to know right now about the 201...,Patrick,Morrisey,CNN,1
5,"[Washington (CNN), Democratic West Virginia Se...","[Updated 5:32 PM ET, Mon August 7, 2017 ]",https://www.cnn.com/2017/08/07/politics/joe-ma...,[Joe Manchin: 'Don't give a s--t' about attacks],Patrick,Morrisey,CNN,1
6,"[Washington (CNN), President Donald Trump's Fr...","[Updated 12:54 PM ET, Fri March 23, 2018 ]",https://www.cnn.com/2018/03/23/politics/trump-...,[Trump's veto threat on spending bill gives am...,Patrick,Morrisey,CNN,1
7,"[Washington (CNN), Several Republican candidat...","[Updated 8:32 PM ET, Wed January 3, 2018 ]",https://www.cnn.com/2018/01/03/politics/steve-...,[Some Bannon-backed Senate hopefuls begin dist...,Patrick,Morrisey,CNN,1
8,"[Washington (CNN), Democratic incumbents, chal...","[Updated 6:06 AM ET, Mon February 5, 2018 ]",https://www.cnn.com/2018/02/05/politics/senate...,[The 10 Senate seats most likely to flip in 20...,Patrick,Morrisey,CNN,1
9,"[ (CNN), The West Virginia Supreme Court of Ap...","[Updated 3:26 PM ET, Fri May 12, 2017 ]",https://www.cnn.com/2017/05/12/us/west-virgini...,[Court: West Virginia hate crime law doesn't i...,Patrick,Morrisey,CNN,1


In [13]:
for index,row in first_data.iterrows():
    print(row["articles_link"], row["newspaper_name"], row["last_name"])

https://www.cnn.com/2017/04/27/politics/joe-manchin-trump-hundred-days/index.html CNN Morrisey
https://www.cnn.com/videos/bestoftv/2014/01/15/exp-costello-morrisey-wv-water.cnn CNN Morrisey
https://www.cnn.com/2014/05/16/us/west-virginia-school-controversy/index.html CNN Morrisey
https://www.cnn.com/2014/01/14/us/west-virginia-contaminated-water/index.html CNN Morrisey
https://www.cnn.com/2018/03/06/politics/midterm-elections-calendar-2018/index.html CNN Morrisey
https://www.cnn.com/2017/08/07/politics/joe-manchin-west-virginia/index.html CNN Morrisey
https://www.cnn.com/2018/03/23/politics/trump-tweet-spending-bill-senate-republican-primaries/index.html CNN Morrisey
https://www.cnn.com/2018/01/03/politics/steve-bannon-senate-candidates-trump-feud/index.html CNN Morrisey
https://www.cnn.com/2018/02/05/politics/senate-race-rankings-february/index.html CNN Morrisey
https://www.cnn.com/2017/05/12/us/west-virginia-supreme-court-hate-crime-ruling/index.html CNN Morrisey
https://www.cnn.com/

https://www.cnn.com/2018/01/22/politics/ted-cruz-shutdown-2013-susan-collins/index.html CNN Cruz
https://www.cnn.com/2018/01/22/politics/senate-vote-reopen-government/index.html CNN Cruz
https://www.cnn.com/2017/08/09/politics/competitive-senate-races-2018-midterms/index.html CNN O'Rourke
https://www.cnn.com/2018/03/18/opinions/white-house-cabinet-shakeup-opinion-stewart/index.html CNN Cruz
https://www.cnn.com/2018/03/29/politics/poll-2020-trump-democrats/index.html CNN Cruz
https://www.cnn.com/2018/03/21/politics/trump-campaign-cambridge-analytica/index.html CNN Cruz
https://www.cnn.com/2017/11/10/politics/house-retirement-tracker/index.html CNN O'Rourke
https://www.cnn.com/2017/12/28/politics/joe-kennedy-seth-moulton-democratic-party/index.html CNN O'Rourke
https://www.cnn.com/2018/01/03/politics/2018-senate-race-rankings-january/index.html CNN O'Rourke
https://www.cnn.com/2018/01/12/politics/white-house-immigration-meeting-conservatives/index.html CNN Cruz
https://www.cnn.com/2018/0

In [ ]:
a = "hello"
a[:4]